Mestrado em Modelagem Matematica da Informacao
----------------------------------------------
Disciplina: Modelagem e Mineracao de Dados
------------------------------------------

Master Program - Mathematical Modeling of Information
-----------------------------------------------------
Course: Data Mining and Modeling
--------------------------------

Professor: Renato Rocha Souza
-----------------------------

### Accessing and Manipulating Data in different formats

In [1]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd

In [2]:
datapath = "/home/rsouza/Dropbox/Renato/ModMinDados/Git/datasets/"

## Acessing CSV Files

In [3]:
csvfile = "beatles-diskography.csv"
datafile = os.path.join(datapath, csvfile)

#### Using pure Python:

In [4]:
data = []
with open(datafile, "rb") as f:
    keys = f.readline().split(',')
    keys = [k.strip() for k in keys]
    for i in range(5):
        values = f.readline().split(',')
        values = [v.strip() for v in values]
        d = dict(zip(keys,values))
        data.append(d)
data[0:2]

TypeError: a bytes-like object is required, not 'str'

#### Using CSV module:

https://docs.python.org/2/library/csv.html  

In [ ]:
import csv

In [ ]:
with open(datafile, "rb") as f:
    #data2 = csv.reader(f)
    data2 = csv.DictReader(f)
    for row in data2:
        print(row)

#### Using Pandas:

In [ ]:
df_csv = pd.read_csv(datafile)
df_csv.head()
#df_csv.info()

## Manipulating Zip Files

https://docs.python.org/2/library/zipfile.html  

In [ ]:
from zipfile import ZipFile

In [ ]:
zipfile = "2013_ERCOT_Hourly_Load_Data"

In [ ]:
with ZipFile('{0}.zip'.format(os.path.join(datapath,zipfile)), 'r') as myzip:
    myzip.extractall()

In [ ]:
myzip.filename

## Acessing Excel Files

https://pypi.python.org/pypi/xlrd  

In [ ]:
import xlrd

In [ ]:
datafile = "2013_ERCOT_Hourly_Load_Data.xls"

In [ ]:
workbook = xlrd.open_workbook(os.path.join(datapath,datafile))
sheet = workbook.sheet_by_index(0)

In [ ]:
sheet_data = [[sheet.cell_value(r, col) for col in range(sheet.ncols)] for r in range(sheet.nrows)]
sheet_data[:2]

In [ ]:
print "Number of rows in the sheet:",
print sheet.nrows

print "Type of data in cell (row 3, col 2):", 
print sheet.cell_type(3, 2)

print "Value in cell (row 3, col 2):", 
print sheet.cell_value(3, 2)

print "Get a slice of values in column 3, from rows 1-3:",
print sheet.col_values(3, start_rowx=1, end_rowx=4)

In [ ]:
coast = sheet.col_values(1, start_rowx=1)
data3 = {}    
data3['maxvalue'] = max(coast)
data3['minvalue'] = min(coast)
data3['avgcoast'] = np.mean(coast)

rowmax = coast.index(max(coast))+1
rowmin = coast.index(min(coast))+1

data3['maxtime'] = xlrd.xldate_as_tuple(sheet.cell_value(rowmax,0), 0)
data3['mintime'] = xlrd.xldate_as_tuple(sheet.cell_value(rowmin,0), 0)

In [ ]:
data3

#### Using Pandas

http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.excel.read_excel.html  

In [ ]:
pd_excel = pd.read_excel(os.path.join(datapath,datafile))
pd_excel.head()

## Accessing HTML

https://docs.python.org/2/library/urllib.html  
https://docs.python.org/2/library/urllib2.html  
http://www.crummy.com/software/BeautifulSoup/  
http://docs.python-requests.org/en/latest/  
http://lxml.de/  
https://docs.python.org/2/library/getpass.html  

In [ ]:
import urllib2
from bs4 import BeautifulSoup as bs
import requests
import lxml.html
import getpass

In [ ]:
url = urllib2.urlopen("http://www.hotashtanga.com/p/letoltesek-downloads.html").read()
soup = bs(url)
for line in soup.find_all('a'):
    print(line.get('href'))

Another approach (some say are faster)

In [ ]:
connection = urllib2.urlopen('http://www.hotashtanga.com/p/letoltesek-downloads.html')
dom =  lxml.html.fromstring(connection.read())
for link in dom.xpath('//a/@href'): # select the url in href for all a tags(links)
    print link

Using Requests, for more complex tasks  

http://www.w3.org/Protocols/rfc2616/rfc2616-sec10.html  

In [ ]:
p = getpass.getpass()
r = requests.get('https://api.github.com/user', auth=('rsouza', p))
r.status_code

In [ ]:
r.headers['content-type']

In [ ]:
r.encoding

In [ ]:
r.text

In [ ]:
r.json()

## Accessing JSON

http://json.org/  
https://docs.python.org/2/library/json.html  
https://docs.python.org/2/library/urlparse.html  

In [ ]:
import json
import urlparse

In [ ]:
BASE_URL = "http://musicbrainz.org/ws/2/"
ARTIST_URL =  urlparse.urljoin(BASE_URL, 'artist/')
query_type = {"simple": {},
              "atr": {"inc": "aliases+tags+ratings"},
              "aliases": {"inc": "aliases"},
              "releases": {"inc": "releases"}}

In [ ]:
def query_site(url, params, uid="", fmt="json"):
    params["fmt"] = fmt
    r = requests.get(url + uid, params=params)
    print "requesting", r.url

    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()

In [ ]:
def query_by_name(url, params, name):
    params["query"] = "artist:" + name
    return query_site(url, params)

In [ ]:
def pretty_print(data, indent=4):
    if type(data) == dict:
        print json.dumps(data, indent=indent, sort_keys=True)
    else:
        print data

In [ ]:
results = query_by_name(ARTIST_URL, query_type["simple"], "Nirvana")
pretty_print(results)

In [ ]:
results.keys()

In [ ]:
artist_id = results["artists"][1]["id"]
artist_id

In [ ]:
print "\nARTIST:"
pretty_print(results["artists"][0])

In [ ]:
artist_data = query_site(ARTIST_URL, query_type["releases"], artist_id)
releases = artist_data["releases"]
print "\nONE RELEASE:"
pretty_print(releases[0], indent=2)

In [ ]:
release_titles = [r["title"] for r in releases]
print "\nALL TITLES:"
for t in release_titles:
        print t

#### Using Pandas

http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader  

In [ ]:
df_json = pd.read_json('https://api.github.com/repos/pydata/pandas/issues?per_page=5')
df_json

## Accessing XML

https://docs.python.org/2/library/urllib2.html

https://docs.python.org/2/library/xml.etree.elementtree.html

http://lxml.de/

http://www.w3schools.com/xml/xml_examples.asp

In [ ]:
from xml.dom import minidom
#from xml.etree import ElementTree as ET
from lxml import etree as ET #Supports xpath syntax
from collections import defaultdict
from pprint import pprint

In [ ]:
BASE_URL = 'http://www.w3schools.com/xml/cd_catalog.xml'
xml_page = urllib2.urlopen(BASE_URL)
str_page = xml_page.read()
e = ET.XML(str_page)

In [ ]:
def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.iteritems():
                dd[k].append(v)
        d = {t.tag: {k:v[0] if len(v) == 1 else v for k, v in dd.iteritems()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v) for k, v in t.attrib.iteritems())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
              d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

In [ ]:
pprint(etree_to_dict(e))

In [ ]:
xml_page = urllib2.urlopen(BASE_URL)
doc = minidom.parse(xml_page)

def findTextnodes(nodeList):
    for subnode in nodeList:
        if subnode.nodeType == subnode.ELEMENT_NODE:
            print("Element node: " + subnode.tagName)
            findTextnodes(subnode.childNodes)
        elif subnode.nodeType == subnode.TEXT_NODE:
            print("text node:" + subnode.data)
            
findTextnodes(doc.childNodes)

In [ ]:
root = ET.fromstring(str_page)
cdtags = root.xpath('//CD/TITLE')
for cd in cdtags:
    print cd.text

In [ ]:
for cd in root.findall('CD'):
    title = cd.find('TITLE').text
    artist = cd.find('ARTIST').text
    country = cd.find('COUNTRY').text
    print('{}\t{}\t{}').format(artist, country, title)

In [ ]:
cds = []
for cd in root.findall('./CD'):
        data = {}
        data["TITLE"] = cd.find('./TITLE').text
        data["ARTIST"] = cd.find('./ARTIST').text
        data["COUNTRY"] = cd.find('./COUNTRY').text
        data["PRICE"] = cd.find('./PRICE').text
        data["YEAR"] = cd.find('./YEAR').text
        cds.append(data)
cds[0:3]

## Acessing Databases

### Acessing SQLite

https://docs.python.org/2/library/sqlite3.html

In [ ]:
import sqlite3
conn = sqlite3.connect('example.db')

In [ ]:
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE stocks
             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [ ]:
conn = sqlite3.connect('example.db')
c = conn.cursor()

In [ ]:
t = ('RHAT',) #tuple with just one element
c.execute('SELECT * FROM stocks WHERE symbol=?', t)
print c.fetchone()

In [ ]:
# Larger example that inserts many records at a time
purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ]
c.executemany('INSERT INTO stocks VALUES (?,?,?,?,?)', purchases)

In [ ]:
for row in c.execute('SELECT * FROM stocks ORDER BY price'):
        print row

### Acessing MySQL  

https://pypi.python.org/pypi/PyMySQL  

In [ ]:
import pymysql

In [ ]:
p = getpass.getpass()
conn = pymysql.connect(host='localhost', port=3306, user='rsouza', passwd=p, db='mysql')
cur = conn.cursor()

In [ ]:
cur.execute("show databases;")
for r in cur.fetchall():
   print(r)

In [ ]:
cur.execute("use mysql;")
cur.execute("show tables;")
for r in cur.fetchall():
   print(r)

In [ ]:
cur.execute("SELECT * FROM mysql.user;")# LIMIT 10;")
for r in cur.fetchall():
   print(r)

#### Using Pandas

In [ ]:
import pandas.io.sql as psql

In [ ]:
df_mysql = psql.read_sql('select * from reuters.Frequency;', con=conn)

In [ ]:
df_mysql.head()

In [ ]:
cur.close()
conn.close()

### Acessing Mongo DB

http://api.mongodb.org/python/current/tutorial.html

In [ ]:
from pymongo import MongoClient

In [ ]:
client = MongoClient()
#client = MongoClient('localhost', 27017)
db = client.test_database #acessa ou cria o banco

In [ ]:
post = {"author": "Mike",
        "text": "My first blog post!",
        "tags": ["mongodb", "python", "pymongo"],
        "date": datetime.datetime.utcnow()}

In [ ]:
posts = db.posts # cria a coleção
post_id = posts.insert(post) #insere dados
post_id

In [ ]:
db.collection_names()

In [ ]:
posts.find_one()

In [ ]:
new_posts = [{"author": "Mike",
              "text": "Another post!",
              "tags": ["bulk", "insert"],
              "date": datetime.datetime(2009, 11, 12, 11, 14)},
             {"author": "Eliot",
              "title": "MongoDB is fun",
              "text": "and pretty easy too!",
              "date": datetime.datetime(2009, 11, 10, 10, 45)}]

In [ ]:
posts.insert(new_posts)

In [ ]:
posts.find_one({"author": "Mike"})

In [ ]:
#for post in posts.find({"author": "Mike"}):
for post in posts.find():
    print(post)

In [ ]:
posts.count()

#### Using Pandas

In [ ]:
example_pd = posts.find()
df_pandas =  pd.DataFrame(list(example_pd))
df_pandas

## Accessing Feeds RSS

http://www.feedparser.org/  
http://docs.python.org/library/re.html  
http://www.pythonware.com/library/pil/handbook/index.htm  

In [ ]:
import feedparser

In [ ]:
#d = feedparser.parse('http://g1.globo.com/dynamo/rss2.xml')
d = feedparser.parse('http://rss.nytimes.com/services/xml/rss/nyt/InternationalHome.xml')

In [ ]:
print d['feed']['title']
print d['feed']['link']
print d.feed.subtitle
print len(d['entries'])

In [ ]:
print d['entries'][0]['title'] 
print d.entries[0]['link']

In [ ]:
print d.headers

In [ ]:
for post in d.entries:
    print post.title + ": " + post.link + "\n"

## Acessando Twitter

https://github.com/bear/python-twitter  
Go to http://twitter.com/apps/new to create an app and get these items  
See https://dev.twitter.com/docs/auth/oauth for more information on Twitter's OAuth implementation  
https://dev.twitter.com/rest/reference/get/account/verify_credentials  

In [ ]:
import twitter

In [ ]:
with open('twitter_tokens.txt', 'r') as twitter_tokens:
    tokens = twitter_tokens.read().split(',')
consumer_key = tokens[0].strip()
consumer_secret = tokens[1].strip()
access_token = tokens[2].strip()
access_token_secret = tokens[3].strip()

Construct, sign, and open a twitter request using the hard-coded credentials above.

In [ ]:
api = twitter.Api(consumer_key=consumer_key, consumer_secret=consumer_secret, 
                  access_token_key=access_token, access_token_secret=access_token_secret)

In [ ]:
print api.VerifyCredentials()

In [ ]:
users = api.GetFriends()
print [u.name for u in users]

In [ ]:
followers = api.GetFollowers()
print [u.screen_name for u in followers]

In [ ]:
mentions = api.GetMentions()
print [m.GetText() for m in mentions]

In [ ]:
statuses = api.GetUserTimeline(screen_name='TIME')
print [s.text for s in statuses]